In [ ]:
import numpy as np
import pickle

from scipy.sparse import rand
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.kernel_ridge import KernelRidge
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix

import implementations as imp
import proj1_helpers as ph

In [ ]:
glove = np.load('embeddings.npy')
y = pickle.load(open('cooc.pkl', 'rb'))
vocab = pickle.load(open('vocab.pkl', 'rb'))

In [ ]:
glove.shape

In [ ]:
print(glove[1])

In [ ]:
thibault = list(zip(y.row, y.col, y.data))

In [ ]:
inv_vocab = {v: k for k, v in vocab.items()}

In [ ]:
print(list(map(lambda t : [inv_vocab[t[0]], inv_vocab[t[1]], t[2]], thibault[:100])))

In [ ]:
def split_id_tweet (list_tweets) :
    ids = [x.split(',')[0] for x in list_tweets]
    tweets =  [','.join(x.split(',')[1:]) for x in list_tweets]
    return ids , tweets

In [ ]:
tweets = []
smile = []
with open('twitter-datasets/train_neg_full.txt', encoding='UTF-8') as f :
    ids , read = split_id_tweet(f.readlines())
    smile = [-1] * len(read)
    tweets_train = read
    
with open('twitter-datasets/train_pos_full.txt', encoding='UTF-8') as f :
    ids , read = split_id_tweet(f.readlines())
    smile += [1] * len(read)
    tweets_train += read
    


In [ ]:
len(tweets_train)


In [ ]:
size_vector = len(vocab)
def to_vector (line) :
    res= [0] * size_vector
    for word in line.split(' ') :
        change_nb = vocab.get(word)
        if change_nb != None :
            res[change_nb] = 1
        
    return res
        
#tweets_res = [ np.array(to_vector(line)).dot(glove) for line in tweets]


In [ ]:
def to_three (idx , line ) :
    res_set= []
    res = []
    for word in line.split(' ') :
        change_nb = vocab.get(word)
        if change_nb != None :
            res_set += [change_nb]
    for change_nb in set(res_set) :
        res += [[1 , idx , change_nb]]
    return res
#tweets_to_sparse = [ np.array(elem) for idx , line in enumerate(tweets,0) for elem in to_three(idx , line)]

def columns ( matrix , i) :
    return [x[i] for x in matrix]


def tweet_to_matrix ( tweets ) :
    tweets_to_sparse = [ np.array(elem) for idx , line in enumerate(tweets,0) for elem in to_three(idx , line)]
    
    sparse_tweets = coo_matrix((columns(tweets_to_sparse,0) , (columns(tweets_to_sparse,1),columns(tweets_to_sparse,2))) , shape=(len(tweets), len(glove)))
    return sparse_tweets.dot(glove)

In [ ]:
#sparse_tweets = coo_matrix((columns(0) , (columns(1),columns(2))) , shape=(len(tweets), len(glove)))
#tweets_res = sparse_tweets.dot(glove)
tweets_res = tweet_to_matrix(tweets_train)


In [ ]:
count = 0
count2 = 0
for x in tweets_res :
    count2 +=1
    if  np.count_nonzero(x) == 0 :
        count +=1
count , len(tweets_res) , count/len(tweets_res) , count2

In [ ]:
w,loss = imp.least_squares(np.array(smile), tweets_res)

In [ ]:
w

In [ ]:
ids_test=[]
tweets_test=[]
with open('twitter-datasets/test_data.txt') as f :
    ids , tweets_test = split_id_tweet(f.readlines())
    
tweets_res_test = tweet_to_matrix(tweets_test)


In [ ]:
count = 0
count2 = 0
for x in tweets_res_test :
    count2 +=1
    if  np.count_nonzero(x) == 0 :
        count +=1
count , len(tweets_res_test) , count/len(tweets_res_test) , count2

In [ ]:
res = tweets_res_test.dot(w)
res = np.sign(res)

In [ ]:
imp.create_csv_submission(ids,res,'results.csv')

In [ ]:
count = 0
for x in res :
    if x  == 0 :
        count +=1
count , len(res) , count/len(res)